In [17]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [18]:
%run -i data_generation/simple_linear_regression.py
%run -i data_preparation/v0.py
%run -i model_configuration/v0.py

In [19]:
%run -i model_training/v0.py

OrderedDict([('0.weight', tensor([[1.9690]], device='cuda:0')), ('0.bias', tensor([1.0235], device='cuda:0'))])


In [20]:
def make_train_step_fn(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def perform_train_step_fn(x, y):
        # Sets model to TRAIN mode
        model.train()
        
        # Step 1 - Computes our model's predicted output - forward pass
        yhat = model(x)
        # Step 2 - Computes the loss
        loss = loss_fn(yhat, y)
        # Step 3 - Computes gradients for both "a" and "b" parameters
        loss.backward()
        # Step 4 - Updates parameters using gradients and the learning rate
        optimizer.step()
        optimizer.zero_grad()
        
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return perform_train_step_fn


In [21]:
%run -i model_configuration/v1.py

In [22]:
train_step_fn

<function __main__.make_train_step_fn.<locals>.perform_train_step_fn(x, y)>

In [23]:
%run -i model_training/v1.py


In [24]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]], device='cuda:0')), ('0.bias', tensor([1.0235], device='cuda:0'))])


In [25]:
train_data = TensorDataset(x_train_tensor, y_train_tensor)
print(train_data[0])


(tensor([0.7713], device='cuda:0'), tensor([2.4745], device='cuda:0'))


In [26]:
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

In [27]:
next(iter(train_loader))


[tensor([[0.2809],
         [0.3253],
         [0.1560],
         [0.5924],
         [0.0651],
         [0.8872],
         [0.4938],
         [0.0055],
         [0.1409],
         [0.0885],
         [0.1849],
         [0.7290],
         [0.8662],
         [0.3117],
         [0.6842],
         [0.1987]], device='cuda:0'),
 tensor([[1.5846],
         [1.8057],
         [1.2901],
         [2.1687],
         [1.1559],
         [2.8708],
         [1.9060],
         [1.0632],
         [1.1211],
         [1.0708],
         [1.5888],
         [2.4927],
         [2.6805],
         [1.7637],
         [2.3492],
         [1.2654]], device='cuda:0')]

In [29]:
%run -i data_preparation/v1.py

In [30]:
%run -i model_training/v2.py


In [35]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9698]], device='cuda:0')), ('0.bias', tensor([1.0246], device='cuda:0'))])


In [36]:
def mini_batch(device, data_loader, step_fn):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_losses)
    return loss


In [37]:
%%writefile model_training/v3.py

# Defines number of epochs
n_epochs = 200

losses = []

for epoch in range(n_epochs):
    # inner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)


Writing model_training/v3.py


In [38]:
%run -i model_training/v3.py


In [39]:
# Checks model's parameters
print(model.state_dict())


OrderedDict([('0.weight', tensor([[1.9682]], device='cuda:0')), ('0.bias', tensor([1.0231], device='cuda:0'))])


In [3]:
import numpy as np

true_b = 1
true_w = 2
N = 100

# Data Generation
np.random.seed(42)
x = np.random.rand(N, 1)
y = true_b + true_w * x + (.1 * np.random.randn(N, 1))

In [7]:
## This is to display things from TensorBoard 

%load_ext tensorboard
%tensorboard --logdir runs

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

def plot_losses(losses, val_losses):
    fig = plt.figure(figsize=(10, 4))
    plt.plot(losses, label='Training Loss', c='b')
    plt.plot(val_losses, label='Validation Loss', c='r')
    plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    return fig


def plot_resumed_losses(saved_epoch, saved_losses, saved_val_losses, n_epochs, losses, val_losses):
    range_before = range(0, saved_epoch)
    range_after = range(saved_epoch, saved_epoch + n_epochs)

    fig = plt.figure(figsize=(10, 4))
    # Checkpointed loss
    plt.plot(range_before, saved_losses, 
             label='Checkpointed Training Loss', c='b', linestyle='--')
    plt.plot(range_before, saved_val_losses, 
             label='Checkpointed Validation Loss', c='r', linestyle='--')
    # Losses after resuming
    plt.plot(range_after, losses, label='Training Loss', c='b')
    plt.plot(range_after, val_losses, label='Validation Loss', c='r')
    # Divider
    plt.plot([saved_epoch, saved_epoch],
             [np.min(saved_losses + losses), np.max(saved_losses + losses)], 
             c='k', linewidth=1, linestyle='--', label='Checkpoint')
    plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    return fig


fig = plot_resumed_losses(saved_epoch, saved_losses, saved_val_losses, n_epochs, losses, val_losses)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 19992), started 0:01:10 ago. (Use '!kill 19992' to kill it.)

NameError: name 'saved_epoch' is not defined